In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

In [3]:
data = pd.read_csv('/hpc/dla_lti/araja/hapsnew/ERAP2/ERAP2haploCLinicalmerged.csv')
data = data.loc[:, (data != 0).any(axis=0)]

In [4]:
data['VTE'].value_counts()


VTE
0    321100
1     12880
Name: count, dtype: int64

In [5]:
selected_columns = data.columns[28:]
counts = {"Column Name": [], "Cases (1 count)": []}
for col in selected_columns:
    counts["Column Name"].append(col)
    counts["Cases (1 count)"].append((data[col] == 1).sum())  
count_table = pd.DataFrame(counts)

In [6]:
data=data.rename(columns={"Sex.0.0": "Sex"})

In [ ]:
####Running logistic regression to find associations of haps with phentoypes####
haplotype_columns = data.columns[1:28]
clinical_feature_columns = data.columns[28:]

X_covariates = sm.add_constant(data[['Sex', 'Age']])
with open('ERAP2hapsClinical.csv', 'a') as f:
    for haplotype in haplotype_columns:
        y = data[haplotype]
        for clinical_feature in clinical_feature_columns:
            X = X_covariates.copy()
            X[clinical_feature] = data[clinical_feature]
            try:
                result = sm.Logit(y, X).fit(method='bfgs', maxiter=1000)
            except:
                try:
                    result = sm.Logit(y, X).fit(method='newton', maxiter=1000)
                except Exception as e:
                    print(f"Error occurred for haplotype {haplotype}, clinical feature {clinical_feature}: {e}")
                    continue

            estimates = result.params[1:]
            conf_int = result.conf_int().iloc[1:]
            odds_ratios = np.exp(estimates)
            p_values = result.pvalues[1:]
            corrected_p_values = multipletests(p_values, method='fdr_bh')[1] #Corrected using Benjamini–Hochberg
            for feature, estimate, conf_interval, p_value, odds_ratio, corrected_p_value in zip(X.columns[1:], estimates, conf_int.values, p_values, odds_ratios, corrected_p_values):
                ci_low, ci_high = np.exp(conf_interval)  
                z_value = estimate / result.bse[feature]
                p_adjusted = corrected_p_value
                result_row = pd.DataFrame({'haplotype': [haplotype],
                                            'clinical_feature': [feature],
                                            'estimate': [estimate],
                                            'std_error': [result.bse[feature]],
                                            'z_value': [z_value],
                                            'p_value': [p_value],
                                            'odds_ratio': [odds_ratio],
                                            'ci_low': [ci_low],
                                            'ci_high': [ci_high],
                                            'p_adjusted': [p_adjusted]})
                result_row.to_csv(f, header=False, index=False)

In [ ]:
###Annotating ICD10 ids and calculating case vs control frequency####
significant_results = pd.read_csv("/hpc/dla_lti/araja/hapsnew/ERAP2/ERAP2hapsClinical.csv")
icd = pd.read_csv("/hpc/dla_lti/araja/hapsnew/ERAP2/ICD_dis_name.csv")
merged = significant_results.merge(icd, how='left', left_on='clinical_feature', right_on='term')
merged = merged[~merged['Clinical_outcome_name'].isna()]
merged['Clinical_outcome_name'] = merged['Clinical_outcome_name'].replace("#N/A", np.nan)
merged = merged[~merged['Clinical_outcome_name'].isna()]
merged = merged.drop_duplicates() #Keeping first instance by default
significant_results['haplo_cases'] = np.nan
significant_results['haplo_controls'] = np.nan
significant_results['total_cases'] = np.nan
significant_results['total_controls'] = np.nan
for i, row in significant_results.iterrows():
    haplo_col = row['haplotype']
    clinical_col = row['clinical_feature']
    if haplo_col in data.columns and clinical_col in data.columns:
        haplo_cases = ((data[haplo_col] == 1) & 
                       (data[clinical_col] == 1)).sum()
        haplo_controls = ((data[haplo_col] == 1) & 
                          (data[clinical_col] == 0)).sum()
        total_cases = (data[clinical_col] == 1).sum()
        total_controls = (data[clinical_col] == 0).sum()
        significant_results.at[i, 'haplo_cases'] = haplo_cases
        significant_results.at[i, 'haplo_controls'] = haplo_controls
        significant_results.at[i, 'total_cases'] = total_cases
        significant_results.at[i, 'total_controls'] = total_controls
significant_results['freq_case'] = significant_results['haplo_cases'] / significant_results['total_cases']
significant_results['freq_control'] = significant_results['haplo_controls'] / significant_results['total_controls']
significant_results.csv("Final-ERAP2hapsClinical.csv", sep=",")